In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np
import pandas as pd
import seaborn as sns



In [2]:
titles = []
years = []
imdb_ratings = []
text_muted1 = []
text_muted2 = []
text_muted3 = []
meta_scores = []
director = []
stars = []


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
for pages in np.arange(1,9900, 250):
    if pages == 1:
        response = get(
            'https://www.imdb.com/search/title/?title_type=feature&num_votes=7500,&languages=en&sort=user_rating,desc&count=250&ref_=adv_prv', headers=headers)
    else:
        response = get(
            'https://www.imdb.com/search/title/?title_type=feature&num_votes=7500,&languages=en&sort=user_rating,desc&count=250&start='+str(pages)+'&ref_=adv_nxt', headers=headers)
    
    sleep(randint(8, 15))

    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(
            requests, response.status_code))

    soup = BeautifulSoup(response.text, 'html.parser')

    for item in soup.select('.lister-item'):
        titles.append(item.h3.a.text)

        year = item.h3.find('span', class_='lister-item-year text-muted unbold')
        years.append(year.text if (year is not None) else None)

        text_muted = item.findAll('p', class_='text-muted')
        text_muted1.append(text_muted[0].text if (text_muted is not None) else None)
        text_muted2.append(text_muted[1].text if (text_muted is not None) else None)

        imdb_ratings.append(item.strong.text if (item.strong.text is not None) else None)

        meta_score = item.find('span', class_='metascore')
        meta_scores.append(meta_score.text if (meta_score is not None) else None)

        director.append([x.text for x in d] if (d := item.select('a[href*="_dr_"]')) else None)
        
        stars.append([x.text for x in s] if (s := item.select('a[href*="_st_"]')) else None)

        text_muted = item.find('p', class_='sort-num_votes-visible')
        text_muted3.append(text_muted.text if (text_muted is not None) else None)


In [3]:
df = pd.DataFrame({'titles': titles,
                   'year': years,
                   'imdb_ratings': imdb_ratings,
                   'text_muted1': text_muted1,
                   'text_muted2': text_muted2,
                   'text_muted3': text_muted3,
                   'meta_scores': meta_scores,
                   'director': director,
                   'stars': stars
                   })

df


,titles,year,imdb_ratings,text_muted1,text_muted2,text_muted3,meta_scores,director,stars
0,The Shawshank Redemption,(1994),9.3,\nR\n|\n142 min\n|\n\nDrama \n,\nTwo imprisoned men bond over a number of yea...,"\nVotes:\n2,647,314\n| Gross:\n$28.34M\n",81,[Frank Darabont],"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,(1972),9.2,"\n175 min\n|\n\nCrime, Drama \n",\nThe aging patriarch of an organized crime dy...,"\nVotes:\n1,835,072\n| Gross:\n$134.97M\n",100,[Francis Ford Coppola],"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,The Dark Knight,(2008),9.0,"\nPG-13\n|\n152 min\n|\n\nAction, Crime, Drama...",\nWhen the menace known as the Joker wreaks ha...,"\nVotes:\n2,619,277\n| Gross:\n$534.86M\n",84,[Christopher Nolan],"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,The Lord of the Rings: The Return of the King,(2003),9.0,"\nPG-13\n|\n201 min\n|\n\nAction, Adventure, D...",\nGandalf and Aragorn lead the World of Men ag...,"\nVotes:\n1,823,184\n| Gross:\n$377.85M\n",94,[Peter Jackson],"[Elijah Wood, Viggo Mortensen, Ian McKellen, O..."
4,Schindler's List,(1993),9.0,"\nR\n|\n195 min\n|\n\nBiography, Drama, Histor...",\nIn German-occupied Poland during World War I...,"\nVotes:\n1,341,758\n| Gross:\n$96.90M\n",94,[Steven Spielberg],"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car..."
...,...,...,...,...,...,...,...,...,...
9895,Superbabies: Baby Geniuses 2,(2004),1.5,"\nPG\n|\n88 min\n|\n\nComedy, Family, Sci-Fi ...",\nA group of smart-talking toddlers find thems...,"\nVotes:\n31,108\n| Gross:\n$9.11M\n",9,[Bob Clark],"[Jon Voight, Scott Baio, Vanessa Angel, Skyler..."
9896,Saving Christmas,(2014),1.3,"\nPG\n|\n79 min\n|\n\nComedy, Family ...",\nHis annual Christmas party faltering thanks ...,"\nVotes:\n15,947\n| Gross:\n$2.78M\n",18,[Darren Doane],"[Kirk Cameron, Darren Doane, Bridgette Cameron..."
9897,Foodfight!,(2012),1.3,"\nPG\n|\n91 min\n|\n\nAnimation, Action, Adven...",\nThe evil Brand X joins a supermarket that be...,"\nVotes:\n11,045\n",None,[Lawrence Kasanoff],"[Hilary Duff, Haylie Duff, Charlie Sheen, Eva ..."
9898,Smolensk,(2016),1.2,"\n120 min\n|\n\nDrama, Thriller \n",\nAn inspired story of people affected by the ...,"\nVotes:\n39,697\n",None,[Antoni Krauze],"[Beata Fido, Maciej Póltorak, Aldona Struzik, ..."


In [4]:
df.to_csv('imdb_scraping.csv')  